<a href="https://colab.research.google.com/github/Jazzy-cats/AstraZenecaHackathon/blob/master/notebooks/AstraZeneca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Jazzy-cats/AstraZenecaHackathon.git

Cloning into 'AstraZenecaHackathon'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 80 (delta 5), reused 6 (delta 1), pack-reused 65
Unpacking objects: 100% (80/80), done.
Checking out files: 100% (50/50), done.


In [2]:
!cp -r /content/AstraZenecaHackathon/images_for_preview /content/

In [ ]:
!pip install -U albumentations

     |████████████████████████████████| 122kB 7.6MB/s 
     |████████████████████████████████| 952kB 15.1MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp36-none-any.whl size=65165 sha256=e1928af54cd18d5a6825a74a8583573f7b91ca3733c72ee9dadd309b423f8acf
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
!pip install pylocron

In [ ]:
import os
import glob
import logging
import random

import cv2
import PIL.Image as Image
#from sklearn.preprocessing import PowerTransformer
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
from torchvision.models import resnet152
import albumentations as A
from fastai.vision.models import DynamicUnet
import holocron.models as models  # For UNet++ and UNet+++

# Exploratory Data Analysis

Let's look at the distribution of the pixel values and see if we can transform into normal distribution

In [ ]:
sns.set_style("darkgrid")

In [ ]:
input = glob.glob("/content/images_for_preview/*/input/*") # log
target1 = glob.glob("/content/images_for_preview/*/targets/*A01*")
target2 = glob.glob("/content/images_for_preview/*/targets/*A02*")
target3 = glob.glob("/content/images_for_preview/*/targets/*A03*") # log

def get_mean_and_std(paths, func=None):
    pixel_values = list()
    for i in paths:
        img = cv2.imread(i, -1)
        if func is not None:
            img = func(img)
        pixel_values.append(img.flatten())
    pixel_values = np.concatenate(pixel_values)
    return np.mean(pixel_values), np.std(pixel_values)

def plot_histogram_w_stats(img_path, mean, std, func=None, ax=None):
    img = cv2.imread(img_path, -1)

    if func is not None:
        img = func(img)
    #plt.figure(figsize=(20,6))
    ax.hist(img.ravel(), 200)
    ax.axvline(x=mean,color='red')
    ax.axvline(x=mean+std,color='green')
    ax.axvline(x=mean-std,color='green')
    #plt.show()

def plot_histograms_w_stats(paths, sample_idx=0, transform=None):

    fig, axs = plt.subplots(1,2, figsize=(20,6))
    fig.suptitle(f"Pixel value distribution: {paths[0]}", fontsize=16)
    axs[0].set_title('Original data')
    axs[1].set_title('After transformation')
    axs[0].set_xlabel('Pixel value')
    axs[0].set_ylabel('Counts')
    axs[1].set_xlabel('Pixel value')
    axs[1].set_ylabel('Counts')

    mean, std = get_mean_and_std(paths)
    print(mean, std)
    plot_histogram_w_stats(paths[sample_idx], mean, std, ax=axs[0])

    mean, std = get_mean_and_std(paths, func=transform)
    print(mean, std)
    plot_histogram_w_stats(paths[sample_idx], mean, std, func=transform, ax=axs[1])
    plt.show()
    

log_func = lambda x: np.log(x + 1e-3)

# 1250.8056198505967 1084.9798588268861
# 6.789944517079453 0.8247920659414303 - np.log(x + 1e-3)
plot_histograms_w_stats(input, transform=log_func)

# 378.4063635628563 486.62929332530007
# 5.5090300908038135 0.8687941890567182 - np.log(x + 1e-3)
plot_histograms_w_stats(target1, transform=log_func)

# 988.0271635755419 898.4957144105864
# 27.253351 15.661491 - np.sqrt
plot_histograms_w_stats(target2, transform=np.sqrt)

# 482.04090429532835 254.4028682542436
# 6.054861212228503 0.49684806276264926 - np.log(x + 1e-3)
plot_histograms_w_stats(target3, transform=log_func)


# Creating a PyTorch Dataset and DataLoader for training

In [ ]:
def get_image_metadata(path_to_file):
    """Function for extracting metadata from filename of dataset provided by AstraZeneca.
    
    B02   - well (row B, column 02) 
    T0001 - timepoint (irrelevant for this dataset) 
    F001  - field of view = site 
    L01   - timeline (irrelevant for this datset) 
    A01   - action list number (3 fluorescent + 1 brightfield action) 
    Z01   - 3D z-number (slice number or vertical position) 
    C01   - imaging channel (1 nuclear, 2 lipids, 3 cytoplasm, 4 brightfield)
    """
    string_arr = os.path.basename(path_to_file).split("_")
    row_col = string_arr[-2] # Row and column
    timepoint = string_arr[-1][:5]
    fied_of_view = string_arr[-1][5:9]
    timeline = string_arr[-1][9:12]
    action_list_number = string_arr[-1][12:15]
    z_number_3d = string_arr[-1][15:18]
    imaging_channel = string_arr[-1][18:21]
    return {
        "row_col":       row_col,
        #"timepoint":     timepoint,  # apparently irrelevant
        "field of view": fied_of_view,
        #"timeline": timeline,  # apparently irrelevant
        "action_list_number": action_list_number,
        "z_number_3d": z_number_3d,
        "imaging_channel": imaging_channel,
        "path": path_to_file
    }

In [ ]:
class ExampleDataset(Dataset):

    def __init__(self, dataset_path, crop_size=(256,256), transform=None):
        """Example dataset for sample images for the Astra Zeneca competition
        
        Group by row_col and field of view
        # row_col
        # field of view
         
        Input and Target share these common values:
        - row_col       = sample id? 
        - field of view = amount of zoom

        For identifying INPUT:
        - action_list_number A04
        - imaging_channel    C04
        - z_number_3d        Z01 - Z07

        For identifying TARGET:
        - action_list_number A01 A02 and A03
        - imaging_channel    C01, C02, C03
        - z_number_3d        Z01
        """
        self.dataset_path = dataset_path
        
        dataset_samples = glob.glob(os.path.join(self.dataset_path, "*/*/Assay*"))
        
        dataset_dicts = [get_image_metadata(path) for path in dataset_samples]
        
        # Group all 7 inputs with all 3 respective targets into variable sample
        samples = dict()
        for sample_dict in dataset_dicts:
            sample_key = (sample_dict["row_col"], sample_dict["field of view"])

            if samples.get(sample_key) is None:
                samples[sample_key] = {
                    "input": dict(),
                    "target": dict()
                }

            if sample_dict["action_list_number"] == "A04": # or sample_dict["imaging_channel"] == "C04"
                # Is an input
                z_number_3d = sample_dict["z_number_3d"]
                samples[sample_key]["input"][z_number_3d] = sample_dict["path"]
            else:
                # Is an target
                action_list_number = sample_dict["action_list_number"]
                samples[sample_key]["target"][action_list_number] = sample_dict["path"]

        self.samples = list(samples.values())
        self.crop_size = crop_size
        self.transforms = transform
        
    def __len__(self):
        #return len(self.samples)
        return 200
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Modulo
        idx = idx % len(self.samples)

        sample_dict = self.samples[idx]

        w, h = cv2.imread(sample_dict["input"]["Z01"], -1).shape
        assert self.crop_size[0] <= w
        assert self.crop_size[1] <= h

        # Select random crop
        #c_w, c_h = self.crop_size
        #crop_x = np.random.randint(0, w - c_w)
        #crop_y = np.random.randint(0, h - c_h)

        #input = torch.zeros((7, c_w, c_h))
        #output = torch.zeros((3, c_w, c_h))
        input = torch.zeros((7, w, h))
        output = torch.zeros((3, w, h))
        for i, z_number_3d in enumerate(["Z01", "Z02", "Z03", "Z04", "Z05", "Z06", "Z07"]):
            img_path = sample_dict["input"][z_number_3d]

            img = cv2.imread(img_path, -1)
            #img = img[crop_x:crop_x+c_w, crop_y:crop_y+c_h]

            img = img.astype(np.int32)
            input[i] = torch.Tensor(img)

        for i, action_list_number in enumerate(["A01", "A02", "A03"]):
            img_path = sample_dict["target"][action_list_number]

            img = cv2.imread(img_path, -1)
            #img = img[crop_x:crop_x+c_w, crop_y:crop_y+c_h]

            img = img.astype(np.int32)
            output[i] = torch.Tensor(img)

        if self.transforms:
            for transform in self.transforms:
                input, output = transform(input, output)

        return input, output

x, y = ExampleDataset("/content/images_for_preview/")[0]
print(x.shape, y.shape)

# Creating Data Augmentations
We will be using the following augmentations

Affine transformations
- Horizontal/Vertical flipping
- Rotation 
- Rescale and Crop 
- Shear (Not implemented)

Color transformations
- ??
- Channel shuffle (Not implemented)

Distortion transformations
- Grid distortion (Not implemented)
- Elastic transform (Not implemented)


## List from 1st place winner of 2018 Data Science Bowl
- Clahe, Sharpen, Emboss
- Gaussian Noise
- Color to Gray
- Inverting - we should not have used it, some images were not predicted correctly on stage2 because of this augmentation
- Remapping grayscale images to random color images
- Blur, Median Blur, Motion Blur
- contrast and brightness
- random scale, rotates and flips
- Heavy geometric transformations: Elastic Transform, Perspective
- Transform, Piecewise Affine transforms, pincushion distortion
- Random HSV
- Channel shuffle - I guess this one was very important due to the nature of the data
- Nucleus copying on images. That created a lot of overlapping nuclei.
- It seemed to help networks to learn better borders for overlapping nuclei.

In [ ]:
class RandomRotation:
    """Rotate by one of the given angles."""

    def __init__(self, p=0.5, angles=30):
        self.p = p
        self.angles = angles

    def __call__(self, input, output):
        if random.random() < self.p:
            angle = random.uniform(-self.angles, self.angles)

            for i in range(len(input)):
                values = input[i].getextrema()
                avg = int(sum(values)/len(values))
                input[i] = TF.rotate(input[i], angle, fill=avg)
            for i in range(len(output)):
                values = output[i].getextrema()
                avg = int(sum(values)/len(values))
                output[i] = TF.rotate(output[i], angle, fill=0)

        return input, output


class HorizontalFlipping:
    """Flip horizontally."""

    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, input, output):
        if random.random() < self.p:
            input = TF.hflip(input)
            output = TF.hflip(output)
        return input, output

class VerticalFlipping:
    """Flip vertically."""

    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, input, output):
        if random.random() < self.p:
            input = TF.vflip(input)
            output = TF.vflip(output)
        return input, output

class Normalize:
    """Normalize values."""

    def __init__(self, input_stats, output_stats=None):
        self.input_mean, self.input_std = input_stats
        self.output_mean, self.output_std = output_stats

    def __call__(self, input, output):
        input = (input - self.input_mean)/self.input_std
        output = (output - self.output_mean)/self.output_std
        return input, output

class DistributionTransform:
    """Transform distribution to be normally distributed."""

    def __init__(self):
        pass

    def __call__(self, input, output):
        input = torch.log(input + 1e-3)
        output[0] = torch.log(output[0] + 1e-3)
        output[1] = torch.sqrt(output[1])
        output[2] = torch.log(output[2] + 1e-3)
        return input, output

class ToPIL:
    """
    """
    def __init__(self):
        pass

    def __call__(self, input, output):
        pil_input = list()
        pil_output = list()
        for i in range(len(input)):
            pil_input.append(Image.fromarray(np.array(input[i], dtype=np.uint16), "I;16"))
        for i in range(len(output)):
            pil_output.append(Image.fromarray(np.array(output[i], dtype=np.uint16), "I;16"))
        return pil_input, pil_output

class ToTensor:
    """
    """
    def __init__(self):
        pass

    def __call__(self, input, output):
        torch_input = list()
        torch_output = list()
        for i in input:
            torch_input.append(TF.to_tensor(i))
        for i in output:
            torch_output.append(TF.to_tensor(i))
        torch_input = torch.cat(torch_input, 0)
        torch_output = torch.cat(torch_output, 0)
        return torch_input, torch_output

class Crop:
    """
    """
    def __init__(self, crop_size, ratio=0.2):
        self.crop_size = crop_size
        self.ratio = ratio

    def __call__(self, input, output):

        width, height = input[0].size

        factor = 1 + random.uniform(0, self.ratio)
        crop_width = self.crop_size[1] * factor
        crop_height = self.crop_size[0] * factor

        crop_x = np.random.randint(0, width - crop_width)
        crop_y = np.random.randint(0, height - crop_height)
        
        for idx, img in enumerate(input):
            input[idx] = TF.crop(img, crop_y, crop_x, crop_height, crop_width)

        for idx, img in enumerate(output):
            output[idx] = TF.crop(img, crop_y, crop_x, crop_height, crop_width)
        return input, output

class Resize:
    """
    """
    def __init__(self, crop_size):
        self.crop_size = crop_size

    def __call__(self, input, output):

        new_input = torch.zeros((7, self.crop_size[0], self.crop_size[1]))
        new_output = torch.zeros((3, self.crop_size[0], self.crop_size[1]))
        for idx, img in enumerate(input):
            new_input[idx] = torch.Tensor(cv2.resize(img.float().numpy(), self.crop_size))

        for idx, img in enumerate(output):
            new_output[idx] = torch.Tensor(cv2.resize(img.float().numpy(), self.crop_size))
        return new_input, new_output

In [ ]:
crop = Crop((256,256))
resize = Resize((256,256))
rotation_transform = RandomRotation()
to_pil = ToPIL()
to_tensor = ToTensor()
hflip_transform = HorizontalFlipping()
vflip_transform = VerticalFlipping()
dist_transform = DistributionTransform()

# Without DistributionTransform
input_stats = np.array([1250.8056198505967, 1084.9798588268861])
output_stats = np.array([
    [378.4063635628563,  988.0271635755419, 482.04090429532835],
    [486.62929332530007, 898.4957144105864, 254.4028682542436]
])
output_stats = np.reshape(output_stats, (2,3,1,1))
normalize = Normalize(input_stats, output_stats)

data_transforms = A.Compose([
  to_pil,
  crop,
  rotation_transform,
  to_tensor,
  resize,
  hflip_transform,
  vflip_transform,
  normalize
  ])

# With DistributionTransform 
input_stats = np.array([6.789944517079453, 0.8247920659414303])
output_stats = np.array([
    [5.5090300908038135, 27.253351, 6.054861212228503],
    [0.8687941890567182, 15.661491, 0.49684806276264926]
])
output_stats = np.reshape(output_stats, (2,3,1,1))

normalize = Normalize(input_stats, output_stats)

data_transforms_2 = A.Compose([
  hflip_transform,
  vflip_transform,
  dist_transform,
  normalize
  ])


x, y = ExampleDataset("/content/images_for_preview/", transform=data_transforms)[0]
print(x.shape, y.shape)
print(x.mean(), y.mean())
print(x.max(), y.max())

def post_process(target):
    output_stats = torch.Tensor([
        [378.4063635628563,  988.0271635755419, 482.04090429532835],
        [486.62929332530007, 898.4957144105864, 254.4028682542436]
    ]).float()
    output_stats = output_stats.view(2,1,3,1,1)
    # (batch, channel, width, height)
    target = target*output_stats[1] + output_stats[0]

    return target.float()

def post_process_2(target):
    output_stats = torch.Tensor([
        [5.5090300908038135, 27.253351, 6.054861212228503],
        [0.8687941890567182, 15.661491, 0.49684806276264926]
    ]).float()
    output_stats = output_stats.view(2,1,3,1,1)
    # (batch, channel, width, height)
    target = target*output_stats[1] + output_stats[0]

    target[:,0] = torch.exp(target[:,0]) - 1e-3
    target[:,1] = target[:,1]**2
    target[:,2] = torch.exp(target[:,2]) - 1e-3
    return target.float()


## Test: Confirm Dataset is working

In [ ]:
# Confirm that Dataset is working
fig, ax = plt.subplots(1,7, figsize=(24,4))
fig.suptitle("Training samples", fontsize=40)
names = ["Z1", "Z2", "Z3", "Z4", "Z5", "Z6", "Z7"]
for i, x_sample in enumerate(x):
    ax[i].set_title(names[i])
    ax[i].imshow(x_sample)

fig, ax = plt.subplots(1,3, figsize=(10,4))
fig.suptitle("Target samples", fontsize=40)
names = ["Nuclei", "Adipocytes", "Cytoplasm"]
for i, x_sample in enumerate(y):
    ax[i].imshow(x_sample)
    ax[i].set_title(names[i])

In [ ]:
plt.figure(figsize=(20,10))
plt.hist(x_sample.numpy().ravel(), 200)
plt.show()

In [ ]:
dataset = ExampleDataset("/content/images_for_preview/", transform=data_transforms)
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=0)
x, y = next(iter(dataloader))
print(x.shape, y.shape)

# Training with UNet

To use UNet, we do the following
1. Obtain pretrained UNet model
2. Change input layer to fit 7 channels
3. Change output layer to output 3 channels


## Data Science Bowl 2018 1st winner:
- They suggest these following backbones for best performance: 
  - DPN-92
  - Resnet-152
  - InceptionResnetV2
  - Resnet101

In [ ]:
class UNet(nn.Module):
    """Just a basic UNet model.

    https://github.com/milesial/Pytorch-UNet
    """
    def __init__(self, input_channels=7, output_channels=3):
        super(UNet, self).__init__()

        # Get model from PyTorch hub
        net = torch.hub.load('milesial/Pytorch-UNet', 'unet_carvana')

        # Configure input layer to take 7 channels
        net.inc.double_conv[0] = nn.Conv2d(
            input_channels, 
            net.inc.double_conv[0].out_channels, 
            kernel_size = net.inc.double_conv[0].kernel_size, 
            stride = net.inc.double_conv[0].stride, 
            padding = net.inc.double_conv[0].padding
            )

        # Configure output layer to output 3 channels
        net.outc.conv = nn.Conv2d(
            net.outc.conv.in_channels,
            output_channels,
            kernel_size = net.outc.conv.kernel_size,
            stride = net.outc.conv.stride
        )
        self.net = net
    
    def forward(self, x):
        return self.net(x)

class UNet3P(nn.Module):
    """Just a basic UNet+++ model.

    https://github.com/frgfm/Holocron
    """
    def __init__(self, input_channels=7, output_channels=3):
        super(UNet3P, self).__init__()

        # Get model from Holocron library
        net = models.unet3p(pretrained=False)

        # Configure input layer to take 7 channels
        net.encoders[0][0] = nn.Conv2d(
            input_channels, 
            net.encoders[0][0].out_channels, 
            kernel_size = net.encoders[0][0].kernel_size, 
            stride = net.encoders[0][0].stride, 
            padding = net.encoders[0][0].padding
        )

        # Configure output layer to output 3 channels
        net.classifier = nn.Conv2d(
            net.classifier.in_channels,
            output_channels,
            kernel_size = net.classifier.kernel_size,
            stride = net.classifier.stride
        )
        self.net = net
    
    def forward(self, x):
        return self.net(x)

class UNet_with_backbone(nn.Module):
    def __init__(self, input_channels=7, output_channels=3):
        super(UNet_with_backbone, self).__init__()
        backbone = torchvision.models.resnet152(pretrained=True)
        backbone.conv1 = nn.Conv2d(
            input_channels,
            out_channels = backbone.conv1.out_channels,
            kernel_size = backbone.conv1.kernel_size,
            stride  = backbone.conv1.stride,
            bias  = backbone.conv1.bias
        )
        backbone = nn.Sequential(*list(backbone.children())[:-2])

        # Using Fastai API to construct UNet from any encoders (ResNet152 in this case)
        UNet = DynamicUnet( 
            backbone,        # Backbone
            output_channels, # Number of classes/channels
            (256,256),       # Output shape
            norm_type=None
        )
        self.UNet = UNet

    def forward(self, x):
        return self.UNet(x)




#models.unetp()  UNet+
#models.unetpp() UNet++
#models.unet3p() UNet+++

# Test inference
model = UNet_with_backbone(input_channels=7, output_channels=3)

# PyTorch training loop

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
epochs = 40
global_step = 0
save_cp = False
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model.to(device)
all_A1_train_losses = list()
all_A2_train_losses = list()
all_A3_train_losses = list()
for epoch in range(epochs):
        model.train()

        epoch_loss = 0
        A1_train_losses = list()
        A2_train_losses = list()
        A3_train_losses = list()
        batch_train_losses = list()
        with tqdm(total=len(dataset), desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for inputs, targets in dataloader:

                inputs = inputs.to(device=device, dtype=torch.float32)
                targets = targets.to(device, dtype=torch.float32)
                
                assert not torch.isnan(inputs).any()  # Input
                assert not torch.isnan(targets).any() # targets
                preds = model(inputs)
                assert not torch.isnan(preds).any() # preds
                loss = criterion(preds, targets)
                epoch_loss += loss.item()

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_value_(model.parameters(), 0.1)
                optimizer.step()

                global_step += 1
                pred = post_process(preds.detach().cpu())
                targets = post_process(targets.cpu())
                comp_loss_A1 = criterion(pred[:,0], targets[:,0]).item()
                comp_loss_A2 = criterion(pred[:,1], targets[:,1]).item()
                comp_loss_A3 = criterion(pred[:,2], targets[:,2]).item()
 
                A1_train_losses.append(comp_loss_A1)
                A2_train_losses.append(comp_loss_A2)
                A3_train_losses.append(comp_loss_A3)
                batch_train_losses.append(loss.item())
                pbar.set_postfix(**{'loss (batch)': np.mean(batch_train_losses), "A1": np.mean(A1_train_losses), "A2": np.mean(A2_train_losses), "A3": np.mean(A3_train_losses)})
                pbar.update(inputs.shape[0])

                #if global_step % (len(dataset) // (10 * batch_size)) == 0:
                #    for tag, value in model.named_parameters():
                #        tag = tag.replace('.', '/')
                #    val_score = eval_net(model, val_loader, device)
                #    scheduler.step(val_score)
                #
                #    if model.n_classes > 1:
                #        logging.info('Validation cross entropy: {}'.format(val_score))
                #    else:
                #        logging.info('Validation Dice Coeff: {}'.format(val_score))
        all_A1_train_losses.append(np.mean(A1_train_losses))
        all_A2_train_losses.append(np.mean(A2_train_losses))
        all_A3_train_losses.append(np.mean(A3_train_losses))

        if save_cp:
            try:
                os.mkdir(dir_checkpoint)
                logging.info('Created checkpoint directory')
            except OSError:
                pass
            torch.save(net.state_dict(),
                       dir_checkpoint + f'CP_epoch{epoch + 1}.pth')
            logging.info(f'Checkpoint {epoch + 1} saved !')


In [ ]:
plt.figure(figsize=(20,10))
plt.plot(all_A1_train_losses)
plt.plot(all_A2_train_losses)
plt.plot(all_A3_train_losses)
plt.legend(["A1"])#, "A2", "A3"])
plt.ylim([50000, 850000])
plt.show()

In [ ]:
# Sample inference and visualization
x, y = ExampleDataset("/content/images_for_preview/", transform=data_transforms)[0]

fig, ax = plt.subplots(1,7, figsize=(24,15))
names = ["Z1", "Z2", "Z3", "Z4", "Z5", "Z6", "Z7"]
for i, x_sample in enumerate(x):
    ax[i].set_title(names[i])
    ax[i].imshow(x_sample)

fig, ax = plt.subplots(1,3, figsize=(10,6))
fig.suptitle("Target samples", fontsize=40)
names = ["Nuclei", "Adipocytes", "Cytoplasm"]
for i, x_sample in enumerate(y):
    ax[i].set_title(names[i])
    ax[i].imshow(x_sample)

model.eval().to(device)
preds = model(x.unsqueeze(0).to(device)).cpu()
fig, ax = plt.subplots(1,3, figsize=(10,6))
fig.suptitle("Prediction samples", fontsize=40)
names = ["Nuclei", "Adipocytes", "Cytoplasm"]
for i, x_sample in enumerate(preds[0]):
    ax[i].set_title(names[i])
    ax[i].imshow(x_sample.detach().numpy())
    break